In [2726]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time

# Set seed
np.random.seed(42)
random.seed(42)

# Classes


In [2727]:
class Individual:
  """
  A class that represents an individual solution.

  Attributes:
    chromosome (1D list): The chromosome of the individual in [0,1] interval.
    weights (2D list): The weights of the individual in [-1,1] interval.
    fitness (float): The fitness of the individual.
  """

  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, fitness:float = math.inf):
    self._chromosome = chromosome
    self._weights = weights
    
    if fitness < 0.0:
      raise ValueError('The fitness of an individual cannot be negative.')
    else:
      self._fitness = fitness
  
  def __str__(self):
    return f"""
    Individual:
        Feature-selection: {self._chromosome}
        Weights: {self._weights}
        Fitness: {self._fitness}
    """
  @property
  def chromosome(self):
    return self._chromosome
  
  @property
  def weights(self):
    return self._weights

  @property
  def fitness(self):
    return self._fitness

  @chromosome.setter
  def chromosome(self, chromosome):
    self._chromosome = chromosome

  @weights.setter
  def weights(self, weights):
    self._weights = weights

  @fitness.setter
  def fitness(self,f):
    self._fitness = f

In [2728]:
class Particle(Individual):
  """
  A class that represents an individual solution in Swarm-based problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    best_fitness (float): The best fitness of the individual.
    best_weights_position (numpy.ndarray): The best weight position of the individual.
    best_chromosome_position (numpy.ndarray): The best chromosome position of the individual.
    weights_velocity (numpy.ndarray): The velocity of the weights of the individual.
    chromosome_velocity (numpy.ndarray): The velocity of the chromosome of the individual.  
  """
  
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, fitness:float = math.inf):
    Individual.__init__(self, chromosome, weights, fitness)
    self._best_fitness = fitness # At the beginning, the best fitness is the input fitness
    self._best_weights_position = weights # By default, the best weight position is the starting weight position.
    self._best_chromosome_position = chromosome # By default, the best chromosome position is the starting chromosome position.
    self._weights_velocity = np.zeros(weights.shape) # All swarm's weights velocities are set to 0 at the start
    self._chromosome_velocity = np.zeros(chromosome.shape) # All swarm's chromosome velocities are set to 0 at the start

  def __str__(self):
    return f"""
    Particle:
        Feature-selection: {self.chromosome}
        Weights: {self.weights}
        Fitness: {self.fitness}
        Actual weights velocity: {self._weights_velocity}
        Actual feature-selection velocity: {self._chromosome_velocity}
        Best fitness: {self._best_fitness}
        Best weights: {self._best_weights_position}
        Best feature-selection: {self._best_chromosome_position}
    """

  @property
  def best_fitness(self):
    return self._best_fitness

  @property
  def best_weights_position(self):
    return self._best_weights_position
  
  @property
  def best_chromosome_position(self):
    return self._best_chromosome_position
  
  @property
  def weights_velocity(self):
    return self._weights_velocity

  @property
  def chromosome_velocity(self):
    return self._chromosome_velocity

  @best_fitness.setter
  def best_fitness(self,f):
    self._best_fitness = f

  @best_weights_position.setter
  def best_weights_position(self,w):
    self._best_weights_position = w

  @best_chromosome_position.setter
  def best_chromosome_position(self,c):
    self._best_chromosome_position = c

  @weights_velocity.setter
  def weights_velocity(self,w):
    self._weights_velocity = w

  @chromosome_velocity.setter
  def chromosome_velocity(self,c):
    self._chromosome_velocity = c

In [2729]:
class Population:
  """
  A class that represents a population of individual solutions.

  Attributes:
  -----------
    size (int): The size of the population.
    gene_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
  
  Methods:
  -----------
    insert_best_gene(Individual): Inserts the best individual of the population into the gene_list.
    add_gene_to_list_at_index(Individual): Adds an individual to the gene_list.
  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    self._size = size
    self._best_gene = None

    if is_empty == True:
      self._genes_list = np.empty(size, dtype=Individual)
    else:
      if is_swarm == True:
        self._genes_list = np.full(shape= size, fill_value= create_particle(K,D))
      else:
        self._genes_list = np.full(shape= size, fill_value= create_individual(K,D))
  
  def __str__(self):
    return f"""
    Population:
        Size: {self._size}
        Gene list: {self._genes_list}
        Best gene: {self._best_gene}
    """
  @property
  def size(self):
    return self._size
  
  @property
  def genes_list(self):
    return self._genes_list

  @property
  def best_gene(self):
    return self._best_gene

  @genes_list.setter
  def genes_list(self, genes_list):
    self._genes_list = genes_list
    
  @best_gene.setter
  def best_gene(self,gen):  
    self._best_gene = gen

  # Methods
  def add_gene_to_list_at_index(self,gen, index:int):
    self._genes_list[index] = gen

  def insert_best_gene(self,gen):
    self.best_gene = gen
    self._genes_list[0] = gen  # Insert the best gene at the beginning of the list

In [2730]:
class Swarm(Population):
  """
  A class that represents a population of solutions in Swarm-based problems.

  Attributes:
  -----------
    (same as Population), and additional attributes:

    global_best_fitness (float): The best fitness of the population.
    global_best_weights (numpy.ndarray): The best weight position of the population.
    global_best_chromosome (numpy.ndarray): The best chromosome position of the population.    
  
  Methods:
  -----------

  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    Population.__init__(self, size, K, D, is_empty, is_swarm=True)
    self._global_best_fitness = math.inf
    self._global_best_weights = None
    self._global_best_chromosome = None

  def __str__(self):
    return f"""
    Swarm:
        Size: {self._size}
        Particles list: {self.genes_list}
        Best particle: {self.best_gene}
        Global best fitness: {self._global_best_fitness}
        Global best weights: {self._global_best_weights}
        Global best feature-selection: {self._global_best_chromosome}
    """
  @property
  def global_best_fitness(self):
    return self._global_best_fitness

  @property
  def global_best_weights(self):
    return self._global_best_weights

  @property
  def global_best_chromosome(self):
    return self._global_best_chromosome

  @global_best_fitness.setter
  def global_best_fitness(self,f):
    self._global_best_fitness = f
  
  @global_best_weights.setter
  def global_best_weights(self,w):
    self._global_best_weights = w
  
  @global_best_chromosome.setter
  def global_best_chromosome(self,c):
    self._global_best_chromosome = c

In [2731]:
class Reef:
  """
  A class that represents a population of solutions in Evolutionary-based problems.

  Attributes:
  ----------- 
    size (int): The size of the population.
    free_occupied_rate (float): The rate of occupied individuals in the reef.
    genes_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
    
  Methods:
  -----------
  """

  def __init__(self, size:int, rate:float, K:int, D:int):
    self._size = size
    self._free_occupied_rate = rate
    self._best_coral = None
    self._corals_list = np.full(shape = [size], fill_value = None) # Empty reef

    occupiedHoles = int(size * rate) # Partially occupy the reef
    self._corals_list[:occupiedHoles] = [create_individual(K,D) for _ in range(occupiedHoles)] # Fill the reef with individuals

  def __str__(self):
    return f"""
    Reef:
        Size: {self._size}
        Free-occupied rate: {self._free_occupied_rate}
        Corals list: {self._corals_list}
        Best coral: {self._best_coral}
    """
  
  @property
  def size(self):
    return self._size

  @property
  def free_occupied_rate(self):
    return self._free_occupied_rate
  
  @property
  def corals_list(self):
    return self._corals_list

  @property
  def best_coral(self):
    return self._best_coral
  
  @corals_list.setter
  def corals_list(self,corals_list):
    self._corals_list = corals_list

  @best_coral.setter
  def best_coral(self,coral):
    self._best_coral = coral
  
  # Methods
  def insert_new_larvae_in_hole(self, larvae: Individual, hole_index:int):
    self.corals_list[hole_index] = larvae

  def remove_coral_from_hole(self, hole_index:int):
    self.corals_list[hole_index] = None
  
  def sort_by_fitness(self):
    # Sort corals_list by larvaes fitness in descending order, and if there are None values, put them at the end of the list
    self.corals_list = sorted(self.corals_list, key=lambda x: x.fitness if x is not None else math.inf)


# METHODS


In [2732]:
def sum_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.add(mat1, mat2)

def subtract_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.subtract(mat1, mat2)

def dot_multiply(mat1:np.ndarray, mat2:np.ndarray):
  return np.multiply(mat1, mat2)

def multiply_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.matmul(mat1, mat2)

def dot_divide(mat1:np.ndarray, mat2:np.ndarray):
  return np.divide(mat1, mat2)

In [2733]:
def random_chromosome_generator(K:int):
  return np.random.randint(low= 0, high= 1 + 1, size = K) # 0 or 1

def random_weights_generator(K:int, D:int):
  return np.random.uniform(low= -1, high= 1, size=(K,D)) # -1 to 1

In [2734]:
def create_individual(K:int = 0, D:int = 0):
  return Individual(random_chromosome_generator(K), random_weights_generator(K,D))

def create_particle(K:int = 0, D:int = 0):
  return Particle(random_chromosome_generator(K), random_weights_generator(K,D))  

def create_population(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
  return Population(size,K,D, is_empty) 

def create_swarm(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
  return Swarm(size,K,D, is_empty) 

def create_reef(size:int, rate:float, K:int = 0, D:int = 0):
  return Reef(size, rate, K, D) 

In [2735]:
def internal_reproduction(gen: Individual):
  # Mutate chromosome and weights
  mutate_point = np.random.randint(0, len(gen.chromosome)) # Choose a random point to mutate
  gen.chromosome[mutate_point] = 1 - gen.chromosome[mutate_point] # Mutate the point
  
  # Mutate weights
  total_rows = gen.weights.shape[0] 
  total_columns = gen.weights.shape[1] 
  mutate_column = np.random.randint(0, total_columns) # Choose a random column to mutate
  
  gen.weights[:,mutate_column] = np.random.uniform(-1,1,total_rows) # Mutate the column

In [2736]:
def external_reproduction(father:Individual, mother:Individual):
  crossover_point = np.random.randint(0, len(father.chromosome))
  crossover_column = random.randint(0, len(father.weights[0]))
  
  # Offsprings are created by combining father and mother chromosomes and weights
  offspring1 = Individual( chromosome = np.concatenate((father.chromosome[:crossover_point], mother.chromosome[crossover_point:])), 
                           weights = np.concatenate((father.weights[:,:crossover_column], mother.weights[:,crossover_column:]), axis=1) ) 
  
  offspring2 = Individual( chromosome = np.concatenate((mother.chromosome[:crossover_point], father.chromosome[crossover_point:])), 
                           weights = np.concatenate((mother.weights[:,:crossover_column], father.weights[:,crossover_column:]), axis=1) ) 
                       
  return [offspring1,offspring2] 

In [2737]:
def roulette_wheel_selection(population:Population): 
  # Create an empty population for the chosen individuals of the Roulette Wheel
  roulettePopulation = create_population(population.size, is_empty=True)
  roulettePopulation.insert_best_gene(population.best_gene) # Make sure we don't lose our best gene in the roulette 

  print("... Roulette Wheel Selection ...")
  # Total population fitness (S)
  S = np.sum([individual.fitness for individual in population.genes_list])

  # Population chromosomes' relative probabilities
  rel_prob = [individual.fitness/S for individual in population.genes_list]

  for idx in range(1, roulettePopulation.size): # -1 because we already inserted the best gene
    r = np.random.uniform() 
    # Find the first index for which q_i < r
    for index,individual in enumerate(population.genes_list): 

      r -= rel_prob[index] 
      if r < 0:
        roulettePopulation.add_gene_to_list_at_index(individual, idx)
        break
    # end choose rouletted individual loop
  # end roulette population loop
  return roulettePopulation  

In [2738]:
''' This functions reproduces a population.
    It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
    Returns the input population, updated with the crossover and mutated childs '''

def reproduce_population(population: Population):
  childs = np.empty(shape=0, dtype=Individual) # list to store future offsprings
  crossover_probability = np.random.uniform()
  mutation_probability = np.random.uniform()
  gene_list_size = len(population.genes_list)

  print("... Reproducing population ...")
  for index,individual in enumerate(population.genes_list):
    # Generate a random mother if the crossover probability is met
    if(np.random.random() >= crossover_probability):
      random_mother_index = np.random.randint(0,gene_list_size) # Choose a random mother

      while(random_mother_index == index): # If the mother is the same as the father, choose another one
        random_mother_index = np.random.randint(0,gene_list_size)
      #end while
      
      # Crossover parents
      mother = population.genes_list[random_mother_index]
      offsprings = external_reproduction(individual, mother)

      # Mutate offsprings if probability is over the mutation probability
      rand_value = np.random.random()
      if(rand_value >= mutation_probability): 
        internal_reproduction(offsprings[0])

      if(rand_value >= mutation_probability):
        internal_reproduction(offsprings[1])

      # Add offsprings to the childs list
      childs = np.append(childs, offsprings)
  
  # Extend the population with the new childs
  population.genes_list = np.concatenate( (population.genes_list, childs) )
  return population

In [2739]:
def update_velocities_and_positions_pso(swarm: Swarm, w:float, c1:float, c2:float):
  for index, particle in enumerate(swarm.genes_list): # For each particle in the swarm

    # Velocities
    r1 = np.random.uniform()
    r2 = np.random.uniform()

    # Chromosome loop
    for k in range(len(particle.chromosome)):
      # Update chromosome velocity 
      particle.chromosome_velocity[k] = ( 
                                          (w * particle.chromosome_velocity[k]) +
                                          (c1*r1*(particle.best_chromosome_position[k] - particle.chromosome[k])) +
                                          (c2*r2*(swarm.global_best_chromosome[k] - particle.chromosome[k]))
                                        )

      # Prevent velociy from going out of bounds
      if(particle.chromosome_velocity[k] > 6):
        particle.chromosome_velocity[k] = 6
      elif(particle.chromosome_velocity[k] < -6):
        particle.chromosome_velocity[k] = -6

      velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*particle.chromosome_velocity[k]) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1

      # Update to next position
      if(np.random.uniform() < velocityProbability):
        particle.chromosome[k] = 1
      else:
        particle.chromosome[k] = 0
    # end chromosome loop

    # Weight loop
    for i in range(particle.weights.shape[0]):
      for j in range(particle.weights.shape[1]):

        particle.weights_velocity[i,j] = ( 
                                          (w * particle.weights_velocity[i,j]) +
                                          (c1*r1*(particle.best_weights_position[i,j] - particle.weights[i,j])) +
                                          (c2*r2*(swarm.global_best_weights[i,j] - particle.weights[i,j]))
                                        )
        # Update to next position
        particle.weights[i,j] += particle.weights_velocity[i,j]
    # end weight loop
  # end particle loop
  return swarm

In [2740]:
def split_reef_candidates(candidates:list, fraction:float):
    number_of_candidates = len(candidates)
    split_point = int(number_of_candidates * fraction)

    # Split candidates in two lists
    broadcast_candidates = candidates[:split_point]
    brooding_candidates = candidates[split_point:]
    
    # If broadcast candidates are odd, add one more candidate to the list and remove it from the brooding candidates
    if len(broadcast_candidates) % 2 != 0:
        broadcast_candidates.append(candidates[split_point])
        brooding_candidates.remove(candidates[split_point])

    return [broadcast_candidates, brooding_candidates]

# ELM


In [2741]:
# Computes the fitness for a given Individual using Extreme Learning Machine (ELM) model
def compute_individual_fitness(individual:Individual, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # W
    individualWeights = dot_multiply(individual.weights, individual.chromosome[:, np.newaxis])

    # Feature-selection Bias given Gene's chromosome
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiply_matrixes(trainingX, individualWeights) + BiasTrainingMatrix
    activationTest = multiply_matrixes(testX, individualWeights) + BiasTestMatrix
    
    H_Training = dot_divide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dot_divide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiply_matrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dot_divide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiply_matrixes(inverse, np.transpose(H_Training))
    Beta = multiply_matrixes(Beta, trainingY)

    # Output
    Y_predicted = multiply_matrixes(H_Test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [2742]:
# Computes and udpates the fitnesses of a given Population 
def compute_population_fitness(population: Population, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  print("\n... Computing fitnesses ...")
  for individual in population.genes_list:
    fitness_i = compute_individual_fitness(individual, D, C, trainingX, trainingY, testX, testY)
    individual.fitness = fitness_i # Update fitness

    # Update best gene in Population
    if population.best_gene is None:
      population.best_gene = individual
    else:
      if individual.fitness < population.best_gene.fitness:
        population.best_gene = individual
  # end for

In [2743]:
# Computes and udpates the fitnesses of a given Swarm
def compute_swarm_fitness(swarm: Swarm, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  print("\n...Computing fitnesses...")
  for particle in swarm.genes_list:  
    fitness_i = compute_individual_fitness(particle, D, C, trainingX, trainingY, testX, testY) 
    particle.fitness = fitness_i

    # Update personal bests and global best
    if particle.fitness < particle.best_fitness:
      particle.best_fitness = particle.fitness
      
      if swarm.best_gene is None:
        swarm.best_gene = particle
      else:
        if particle.fitness < swarm.best_gene.fitness:
          swarm.best_gene = particle

  # Set Swarm's best chromosome and weights
  swarm.global_best_chromosome = swarm.best_gene.chromosome
  swarm.global_best_weights = swarm.best_gene.weights
  swarm.global_best_fitness = swarm.best_gene.fitness
  print("Global best fitness: ", swarm.global_best_fitness)

In [2744]:
# Computes and udpates the fitnesses of a given Reef
def compute_reef_fitness(reef: Reef, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  print("\n...Computing fitnesses...")
  for index,coral in enumerate(reef.corals_list):  
    if coral is None: # If the coral is empty, skip it
      continue

    fitness_i = compute_individual_fitness(coral, D, C, trainingX, trainingY, testX, testY) 
    coral.fitness = fitness_i

    # Update best gene in Reef
    if reef.best_coral is None:
      reef.best_coral = coral
    else:
      if coral.fitness < reef.best_coral.fitness:
        reef.best_coral = coral

In [2745]:
# Computes and udpates the fitnesses of a given Pool list
def compute_pool_fitness(pool: list, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    for larvae in pool:  
        fitness_i = compute_individual_fitness(larvae, D, C, trainingX, trainingY, testX, testY) 
        larvae.fitness = fitness_i
    # end for   
    return pool

In [2746]:
def train_model_and_output_results(best_weigths:np.ndarray, best_chromosome:np.ndarray, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # Feature-selected weights
    weights = dot_multiply(best_weigths, best_chromosome[:, np.newaxis]) 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiply_matrixes(trainingX, weights) + BiasTrainingMatrix
    activationTest = multiply_matrixes(testX, weights) + BiasTestMatrix
    
    H_Training = dot_divide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dot_divide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiply_matrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dot_divide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiply_matrixes(inverse, np.transpose(H_Training))
    Beta = multiply_matrixes(Beta, trainingY)

    # Prediction
    predictedLabels = multiply_matrixes(H_Test, Beta)

    correctPrediction = 0
    for i in range(testY.shape[0]):
      if np.round(predictedLabels[i]) == testY[i]:
        correctPrediction +=1
    
    CCR = correctPrediction / testY.shape[0]
    return CCR * 100

# BIO-INSPIRED ALGORITHMS


In [2747]:
''' This functions apply the Evolutionary-based Genetic Algorithm to a given population '''
def ga(population: Population, max_generations:int, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:
    print("- - - - - - - - - - - - - - - - -")
    print("Generation: ", t)
    
    # Reproduction
    reproduce_population(population)

    # Compute Fitness
    compute_population_fitness(population, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Roulette Selection
    population = roulette_wheel_selection(population)

    # Continue iterating
    t += 1
  # end while 
  return population

In [2748]:
''' This method apply the Swarm-based algorithm of PSO to a given swarm'''
def pso(swarm: Swarm, max_generations:int, w_max:float,  w_min:float, c1:float, c2:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:
    print("- - - - - - - - - - - - - - - - -")
    print("Generation: ", t)

    # Linearly decrease the inertia weight
    w = w_max - (w_max - w_min) * t / max_generations

    # Evaluate Particles' Fitness and update Pb_i and Gb
    compute_swarm_fitness(swarm, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)
  
    # Update Velocity and Position of each particle
    update_velocities_and_positions_pso(swarm, w, c1, c2)

    # Increase step
    t += 1

    # Debug
    # print(f"Global best gene fitness {swarm.best_gene.fitness} - at generation {t}")
    # print(f"Global best feature selection {swarm.best_gene.chromosome}")

  # end while
  return swarm

In [2749]:
def larvae_setting(pool:np.ndarray, reef:Reef, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
    # Compute pool fitness
    pool = compute_pool_fitness(pool, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Try to settle the larvaes from the pool
    for larvae in pool:
      random_hole = np.random.randint(0, reef.size) # Random hole in the reef
      
      if reef.corals_list[random_hole] is None: # If the hole is empty, settle the larva
        reef.insert_new_larvae_in_hole(larvae, random_hole)
      elif reef.corals_list[random_hole] is not None and larvae.fitness < reef.corals_list[random_hole].fitness:
        reef.insert_new_larvae_in_hole(larvae, random_hole)
      #else: # If the hole is not empty and the larva is not better than the coral, the larva dies
        # print(f"Discarding larvae {larvae.fitness} in hole {random_hole} which has a coral with fitness {reef.corals_list[random_hole].fitness}")
    # end for
    return reef


In [2750]:
def asexual_reproduction(reef:Reef, f_asexual:float):
    # Sort the coral in ascending order of fitness
    reef.sort_by_fitness()

    # Duplicate the best corals in the reef with a fraction 'f_asexual'
    corals_in_reef = [c for c in reef.corals_list if c is not None]
    asexual_set = corals_in_reef[: int(f_asexual * len([coral for coral in reef.corals_list if coral is not None]))] 

    return np.array(asexual_set) 

In [2751]:
def reproduce_reef(reef:Reef, f_broadcast:float):
  pool = np.empty(shape=0, dtype=Individual) # pool of candidates to select from
  reproduction_candidates = [c for c in reef if c is not None]

  # Split reproduction sets
  [broadcast_set, brooding_set] = split_reef_candidates(reproduction_candidates, f_broadcast)

  # Broadcast    
  if len(broadcast_set) > 0: # If broadcast set is not empty, crossover
    while len(broadcast_set) > 0:
      # Select random father and a mother to crossover
      [father_candidate, mother_candidate] = np.random.choice(broadcast_set, 2, replace=False)
      offsprings = external_reproduction(father = father_candidate, 
                                          mother =  mother_candidate)

      # Add offspring to the pool
      pool = np.append(pool,offsprings[0]) # Two parents reproduce only one coral larva
      
      # Remove fathers of the reproduction list
      broadcast_set.remove(father_candidate)
      broadcast_set.remove(mother_candidate)
  
  #  Brooding
  if len(brooding_set) > 0: # If broadcast set is not empty, mutate every individual
    for c in brooding_set:
      internal_reproduction(c)
      pool = np.append(pool, c) # Add the mutated coral to the pool
  
  return pool

    

In [2752]:
def predation(reef:Reef, f_depredation:float, depredation_probability:float):
  # Sort the coral in ascending order of fitness
  reef.sort_by_fitness()

  occupied_corals_in_reef = np.empty(shape=0, dtype=Individual) 
  empty_corals_in_reef = np.empty(shape=0, dtype=Individual)

  for coral in reef.corals_list:
    if coral is not None:
      occupied_corals_in_reef = np.append(occupied_corals_in_reef, coral)
      #print(f"Coral {coral.chromosome} has fitness {coral.fitness}")
    else:
      empty_corals_in_reef = np.append(empty_corals_in_reef, coral)
      #print("None")

  predation_number = int(f_depredation * len(occupied_corals_in_reef))
  #print("------- Starting predation -------")
  #print(f"Predation number {predation_number}")
  for i in range(1, predation_number+1):
    # Remove the last corals in reef.corals_list if it is not None and if the depression probability is met
    if np.random.uniform() < depredation_probability:  
      #print(f"Deleting coral {occupied_corals_in_reef[-i].fitness}")
      occupied_corals_in_reef = np.delete(occupied_corals_in_reef, -i)
      empty_corals_in_reef = np.append(empty_corals_in_reef, None)

  reef.corals_list = np.append(occupied_corals_in_reef, empty_corals_in_reef)
  return reef

In [2753]:
def cro(reef:Reef, max_generations: int, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  # Evaluate reef at the beginning
  compute_reef_fitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

  t = 0
  while t < max_generations:
    print("- - - - - - - - - - - - - - - - -")
    print("Generation:", t)

    # Update variables
    depredation_probability = 0.2 # predation probability
    f_broadcast = np.random.uniform() # broadcast fraction
    f_asexual = np.random.uniform() # asexual fraction

    f_depredation = f_asexual 
    if (f_depredation + f_asexual > 1):
      f_depredation = f_asexual = 0.5

    # Reproduction
    auxiliar_reef = np.copy(reef.corals_list) # Copy reef to avoid changing the original individuals
    pool = reproduce_reef(auxiliar_reef, f_broadcast)

    # Larvae settings
    reef = larvae_setting(pool, reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Asexual reproduction
    asexual_set = asexual_reproduction(reef, f_asexual)

    reef.corals_list = np.append(reef.corals_list, asexual_set) # Add asexual set to the reef

    # Predation
    reef = predation(reef, f_depredation, depredation_probability)
        
    # Fitness Evaluation
    compute_reef_fitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Increase step
    t += 1
  #end while

  reef.sort_by_fitness() # Final sort
  reef.best_coral = reef.corals_list[0] # Best coral is the first one in the final sorted reef
  return reef

# EXPORTER

In [2754]:
def export(df:pd.DataFrame, filename:str):
    # If the file exists, append the new data
    if os.path.isfile(filename):
        with pd.ExcelWriter(filename, mode='a' ,engine='openpyxl', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name='ELM', index=False)
    else:
        df.to_excel(filename, sheet_name='ELM', index=False)

# MAIN


In [2755]:
# HYPERPARAMETERS FOR RUNNING SCRIPTS
# GENERAL
# K - declared already as the number of features
OPTIMAL_D = 10 
OPTIMAL_C = 1 
MAX_GENERATIONS = 5

POPULATION_SIZE = 10
SWARM_SIZE = 10
REEF_SIZE = 10

# GA
crossover_prob = 0.65 # This value and mutation_prob are randomly set at Reproduce Population function
mutation_prob = 0.15

# PSO
w_max = 1 # inertia weight coefficient (max)
w_min = 0.3 # inertia weight coefficient (min)
c1 = 2 # cognitive coefficient
c2 = 2 # social coefficient

# CRO
rho0 = 0.8 # free/occupied rate
#eta = 2 # larvae's attempts


In [2756]:
if __name__ == "__main__":
    PORTATIL_DEV = False

    if PORTATIL_DEV:
        CSV_PATH = "C:/Users/david/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
    else:
        CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
        
    data_directory = os.listdir(CSV_PATH)

    for csv in data_directory:
        # Read CSV
        df = pd.read_csv(CSV_PATH + csv, sep=" ", header=None)
        end = df.shape[1]

        # DATAFRAME CONSTANTS
        X = df.iloc[:, :end-1]
        Y = df.iloc[:, end-1] 

        X = X.to_numpy() # IMPORTANT, otherwise we would be using DataFrames/Series instead of numpy arrays
        Y = Y.to_numpy()
        J = len(np.unique(Y)) # Number of classes
        N,K = X.shape[0],X.shape[1]  # N = number of samples, K = number of features
        # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

        X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # Scaling X (min-max normalization)
        X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # First partition with 20% test data
        X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # Second partition with 20% validation data
        

        # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #

        # GA
        ga_start_execution_time = time.time()
        first_population = create_population(POPULATION_SIZE, K, OPTIMAL_D)
        lastGeneration = ga(first_population, MAX_GENERATIONS, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
        GA_CCR = train_model_and_output_results(best_weigths = lastGeneration.best_gene.weights, 
                                                best_chromosome = lastGeneration.best_gene.chromosome, 
                                                D = OPTIMAL_D,
                                                C = OPTIMAL_C, 
                                                trainingX = X_train,
                                                trainingY = Y_train,
                                                testX = X_test,
                                                testY = Y_test)
        ga_end_execution_time = time.time() 

        # PSO
        pso_start_execution_time = time.time()
        first_swarm = create_swarm(SWARM_SIZE, K, OPTIMAL_D)  
        final_swarm = pso(first_swarm, MAX_GENERATIONS, w_max, w_min, c1, c2, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
        PSO_CCR = train_model_and_output_results(best_weigths = final_swarm.global_best_weights, 
                                                best_chromosome = final_swarm.global_best_chromosome, 
                                                D = OPTIMAL_D,
                                                C = OPTIMAL_C, 
                                                trainingX = X_train,
                                                trainingY = Y_train,
                                                testX = X_test,
                                                testY = Y_test)
        pso_end_execution_time = time.time()
        

        # CRO
        cro_start_execution_time = time.time()
        reefFirstGeneration = create_reef(REEF_SIZE, rho0, K, OPTIMAL_D)
        last_reef = cro(reefFirstGeneration, MAX_GENERATIONS, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
        CRO_CCR = train_model_and_output_results(best_weigths = last_reef.best_coral.weights, 
                                                best_chromosome = last_reef.best_coral.chromosome, 
                                                D = OPTIMAL_D,
                                                C = OPTIMAL_C, 
                                                trainingX = X_train,
                                                trainingY = Y_train,
                                                testX = X_test,
                                                testY = Y_test)
        cro_end_execution_time = time.time()

        # Export results to Excel
        results_dict = {"Database": csv , "GA CCR": GA_CCR, "PSO CCR": PSO_CCR, "CRO CCR": CRO_CCR, "D": OPTIMAL_D, "C": OPTIMAL_C}
        results_df = pd.DataFrame(results_dict, index=[0])
        export(results_df, "results.xlsx")

        # Debug
        #print("GA CCR:", GA_CCR, "\nError:", 100-GA_CCR)
        #print("GA execution time: ", time.time() - ga_start_execution_time, "sec")
        #print(f"Best solution found in PSO: {final_swarm.best_gene}")
        #print(f"Maximum weight value found: {final_swarm.best_gene.weights.max()} - Minimum weight value found: {final_swarm.best_gene.weights.min()}")
        #print(f"Maximum velocities value found: {final_swarm.best_gene.weights_velocity.max()} - Minimum velocities value found: {final_swarm.best_gene.weights_velocity.min()}")
        #print("PSO CCR:", PSO_CCR, "\nError:", 100-PSO_CCR)
        #print("PSO execution time: ", time.time() - pso_start_execution_time, "sec")
        #print("CRO CCR:", CRO_CCR, "\nError:", 100-CRO_CCR)
        #print("CRO execution time: ", time.time() - cro_start_execution_time, "sec")


- - - - - - - - - - - - - - - - -
Generation:  0
... Reproducing population ...

... Computing fitnesses ...
... Roulette Wheel Selection ...
- - - - - - - - - - - - - - - - -
Generation:  1
... Reproducing population ...

... Computing fitnesses ...
... Roulette Wheel Selection ...
- - - - - - - - - - - - - - - - -
Generation:  2
... Reproducing population ...

... Computing fitnesses ...
... Roulette Wheel Selection ...
- - - - - - - - - - - - - - - - -
Generation:  3
... Reproducing population ...

... Computing fitnesses ...
... Roulette Wheel Selection ...
- - - - - - - - - - - - - - - - -
Generation:  4
... Reproducing population ...

... Computing fitnesses ...
... Roulette Wheel Selection ...
- - - - - - - - - - - - - - - - -
Generation:  0

...Computing fitnesses...
Global best fitness:  28.025124657596542
- - - - - - - - - - - - - - - - -
Generation:  1

...Computing fitnesses...
Global best fitness:  30.455371492059054
- - - - - - - - - - - - - - - - -
Generation:  2

...Com

ValueError: high <= 0